In [1]:
import numpy as np
import scipy
import sympy
import data_import

In [6]:
pd_data = data_import.read_files_to_pd_dataframe(
    [f"../outs/grid/itr_{i}.xml.out" for i in range(500)]
    , tol=1e-06
)

In [7]:
np.alltrue(np.array([1, 2, 0]) >= 0)

True

In [8]:
def eval_lcp_soln(Q, b, x, tol = 1e-05):
    obj_gt_0 = (Q @ x + b >= -tol).all()
    x_gt_0 = (x >= 0).all()
    complementarity = (-tol <= x.T @ (Q @ x + b) <= tol)

    return obj_gt_0 and x_gt_0 and complementarity

def update_value(policy, Q, b, use_cg = True):
    """
    returns the next x given the policy
    """
    I = np.eye(len(b))
    policy_matrix = np.zeros((b.shape[0], b.shape[0]))
    for p in policy:
        policy_matrix[p,p] = 1
    A = np.dot(
            np.dot(
                policy_matrix,
                Q
            ),
            policy_matrix
        ) + I - policy_matrix
    rhs = -np.dot(policy_matrix, b) 
    lst_sqr_soln = np.linalg.lstsq(A , rhs)[0]
    cg_soln = scipy.sparse.linalg.cg(A , rhs , tol=1e-12)[0]
    if use_cg:
        return cg_soln
    else:
        return lst_sqr_soln

def get_Q(pd_data, i):
    return pd_data['Q'][i]

def get_b(pd_data, i):
    CoR = 1
    N = pd_data['N'][i]
    v0 = pd_data['v0'][i]

    return (1 + CoR) * N.T @ v0


In [9]:
corrects = []
for i in range(len(pd_data)):
    Q = get_Q(pd_data, i)
    b = get_b(pd_data, i)
    policy = pd_data['ipopt_policy'][i]

    inner_solver_sol = update_value(policy, Q, b)
    corrects.append(eval_lcp_soln(Q, b, inner_solver_sol))

num_corrects = sum(corrects)
num_incorrect = len(corrects) - num_corrects
print(num_corrects)
print(num_incorrect)

# takeaway: about 20% of the time, using ipopt's solution to reverse-engineer a "policy"
# will not lead to a correct LCP solution by the inner solver

/tmp/ipykernel_10052/3865475345.py:24: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  lst_sqr_soln = np.linalg.lstsq(A , rhs)[0]


410
90


In [11]:
correct_policy_doesnt_work_cases = [i for i in range(len(pd_data)) if corrects[i]]

for i in correct_policy_doesnt_work_cases:

In [10]:
# how many ipopt solns actually satisfy the LCP conditions?
corrects = []
for i in range(len(pd_data)):
    Q = get_Q(pd_data, i)
    b = get_b(pd_data, i)
    ipopt_sol = pd_data['ipopt_sol'][i]
    corrects.append(eval_lcp_soln(Q, b, ipopt_sol))

num_corrects = sum(corrects)
num_incorrect = len(corrects) - num_corrects
print(num_corrects)
print(num_incorrect)

# takeaway: the vast majority of the time, the LCP solution is not being found by IPOPT,
# instead ipopt is just able to optimize the "primal"?

34
466
